In [44]:
import shutil
import os
import glob
import json
folder_0_images = glob.glob("../data/circl-ail-dataset-01/folder_0/*.png")

folder_1_images = glob.glob("../data/circl-ail-dataset-01/folder_1_SECONDPASS/*.png")

label_file = "../data/circl-ail-dataset-01/labels_0.json"
with open(label_file) as f:
    folder_0_labels = json.load(f)
label_file = "../data/circl-ail-dataset-01/labels_1.json"
with open(label_file) as f:
    folder_1_labels = json.load(f)
data_dir = "../data/website_images_1000/"
common_class = "general"


In [45]:
all_train_files = glob.glob(f"{data_dir}/**/*.png", recursive=True)

In [46]:
len(all_train_files)

3000

In [48]:
all_train_filenames = list(map(os.path.basename, all_train_files))

In [23]:
import shutil

data_dir = "../data/test_website_images_25/"
common_class = "general"
label_count = {}
max_count = 25
os.makedirs(os.path.join(data_dir,common_class), exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
labels =  ["marketplace","forum","general"]
all_images=  folder_0_images + folder_1_images
all_lables= folder_0_labels.update(folder_1_labels)
for img_path in all_images:
    filename  = os.path.basename(img_path)
    if filename in all_train_filenames:
        continue
    label_list = folder_0_labels[filename]
    found_label = False
    for label in labels:
        os.makedirs(os.path.join(data_dir,label), exist_ok=True)
        if label in " ".join(label_list) and label_count.get(label, 0) < max_count:
            shutil.copy(img_path, os.path.join(data_dir,label,filename))
            found_label = True
            label_count[label] = label_count.get(label, 0) + 1
            break
    if not found_label and label_count.get(common_class, 0) < max_count:
        label_count[common_class] = label_count.get(common_class, 0 )+ 1
        shutil.copy(img_path, os.path.join(data_dir,common_class,filename))




In [49]:
label_count

{'general': 25, 'marketplace': 25, 'forum': 25}

In [50]:
glob.glob("../data/test_website_images_25/**/*.png", recursive=True)[0]

'../data/test_website_images_25\\forum\\strong-spurious-plausible-database.png'

In [66]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model.fc = nn.Linear(num_ftrs, 3)
data_dir = "../data/test_website_images_25/"
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_dataset = torchvision.datasets.ImageFolder( root=data_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(
	test_dataset,
	batch_size=64,
	shuffle=False,
	num_workers=4
      )


c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [53]:
PATH = './website_classifier.pth'
# torch.save(model.state_dict(), PATH)

In [30]:
# torch.save(model, "./website_classifier_model")

In [67]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
dataiter = iter(test_loader)
images, labels = next(dataiter)
classes = ['forum', 'general', 'marketplace']
# print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [54]:
# net = None
model.load_state_dict(torch.load(PATH))
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  forum marketplace marketplace general


In [68]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 75 test images: {100 * correct // total} %')

RuntimeError: DataLoader worker (pid(s) 27220, 11208, 27072, 16312) exited unexpectedly

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

MemoryError: Caught MemoryError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\datasets\folder.py", line 229, in __getitem__
    sample = self.loader(path)
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\datasets\folder.py", line 268, in default_loader
    return pil_loader(path)
  File "c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\dataset

## Single File inference

In [69]:
from PIL import Image

# Function to load an image and perform the necessary transformations
def process_image(image_path):
    # Load Image
    img = Image.open(image_path).convert("RGB")
    
    # Apply transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    img_t = transform(img)
    
    # Convert to a batch of 1
    img_u = torch.unsqueeze(img_t, 0)
    
    return img_u
image = Image.open('../data/test_website_images_25\\forum\\strong-spurious-plausible-database.png')
image = process_image('../data/test_website_images_25\\forum\\strong-spurious-plausible-database.png')

In [70]:
# net = None

model.load_state_dict(torch.load(PATH))
outputs = model(image)
_, predicted = torch.max(outputs, 1)
classes[predicted[0]]


'marketplace'

In [71]:
classes

['forum', 'general', 'marketplace']

In [61]:
predicted

tensor([2])